# Kohonen

[Some code](https://github.com/giannisnik/som/blob/master/som.py) for SOM in Pytorch

In [ ]:
import torch
import torch.nn as nn
import numpy as np
 

class SOM(nn.Module):
    """
    2-D Self-Organizing Map with Gaussian Neighbourhood function
    and linearly decreasing learning rate.
    """
    def __init__(self, m, n, dim, alpha=None, sigma=None):
        super(SOM, self).__init__()
        self.m = m
        self.n = n
        self.dim = dim
        self.n_neurons = self.m * self.n
        # gain coefficient
        self.alpha = 0.3
        # correction
        self.sigma = max(m, n) / 2.0
        
        if alpha is not None:
            self.alpha = float(alpha)
        if sigma is not None:
            self.sigma = float(sigma)

        self.weights = torch.randn(self.n_neurons, dim)
        self.locations = torch.tensor(np.array(list(self.make_grid_locations_iter(m, n))), dtype=torch.int32)
        self.pdist = nn.PairwiseDistance(p=2)

    def get_weights(self):
        return self.weights

    def get_locations(self):
        return self.locations

    def make_grid_locations_iter(self, m, n):
        for i in range(m):
            for j in range(n):
                yield np.array([i, j])

    def map_vects(self, input_vects):
        to_return = []
        for vect in input_vects:
            min_index = min([i for i in range(self.n_neurons)],
                            key=lambda x: np.linalg.norm(vect-self.weights[x]))
            to_return.append(self.locations[min_index])

        return to_return

    def forward(self, x, learning_rate_op):
        dists = self.pdist(x, self.weights) # compare X with every column in W. columns is a batch dimension
        bmu_index = dists.min(0).indices # find the minimum distance
        bmu_loc = self.locations[bmu_index,:]
        
        alpha_op = self.alpha * learning_rate_op
        sigma_op = self.sigma * learning_rate_op

        diff = self.locations - bmu_loc # .unsqueeze(0).repeat(self.n_neurons, 1) - don't need to copy because broadcasting will do it (ref: https://numpy.org/doc/stable/user/basics.broadcasting.html)
        bmu_distance_squares = torch.sum(torch.pow(diff.float(), 2), 1) # array where for every neuron i: x_i ^2 + y_i ^2 -> d_i
        neighbourhood_func = torch.exp(torch.neg(torch.div(bmu_distance_squares, sigma_op**2))) # e^{ -(d_i / sigma^2) }
        gain_coefficient = alpha_op * neighbourhood_func
        learning_rate_multiplier = gain_coefficient.repeat(self.dim, 1).T # copy to every dimension
        delta = torch.mul(learning_rate_multiplier, x - self.weights) # (x - self.weights) - uses broadcasting too

        self.weights += delta
    
    def forward_no_training(self, x):
        # find a location where the distance between x and m_i is the minimum 
        min_index = torch.linalg.vector_norm(x - self.weights, ord=2, dim=1).min(0).indices
        return self.locations[min_index]



## Example: sort colors

Colors - 3-dimensional vectors. We organize them on 2D grid

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt


#Training inputs for RGBcolors
colors = torch.tensor([
    [0., 0., 0.],
    [0., 0., 1.],
    [0., 0., 0.5],
    [0.125, 0.529, 1.0],
    [0.33, 0.4, 0.67],
    [0.6, 0.5, 1.0],
    [0., 1., 0.],
    [1., 0., 0.],
    [0., 1., 1.],
    [1., 0., 1.],
    [1., 1., 0.],
    [1., 1., 1.],
    [.33, .33, .33],
    [.5, .5, .5],
    [.66, .66, .66]
])
color_names = [
    'black', 'blue', 'darkblue', 'skyblue',
    'greyblue', 'lilac', 'green', 'red',
    'cyan', 'violet', 'yellow', 'white',
    'darkgrey', 'mediumgrey', 'lightgrey'
]

#Train a 20x30 SOM with 100 iterations
m = 20
n = 30
n_iter = 100
som = SOM(m, n, 3)
for iter_no in range(n_iter):
    #Train with each vector one by one
    for i in range(len(colors)):
        learning_rate_op = 1.0 - iter_no/(1.0*n_iter)
        som(colors[i], learning_rate_op)

#Store a centroid grid for easy retrieval later on
centroid_grid = [[] for i in range(m)]
weights = som.get_weights()
locations = som.get_locations()
for i, loc in enumerate(locations):
    centroid_grid[loc[0]].append(weights[i].numpy())

#Get output grid
image_grid = centroid_grid

#Map colours to their closest neurons
mapped = som.map_vects(colors)

#Plot
plt.imshow(image_grid)
plt.title('Color SOM')
for i, m in enumerate(mapped):
    plt.text(m[1], m[0], color_names[i], ha='center', va='center',
             bbox=dict(facecolor='white', alpha=0.5, lw=0))
plt.show()

## Step by step

In [ ]:
diff = som.locations - som.locations[2]
bmu_distance_squares = torch.sum(torch.pow(diff.float(), 2), 1)
alpha_op = som.alpha
sigma_op = som.sigma
neighbourhood_func = torch.exp(torch.neg(torch.div(bmu_distance_squares, sigma_op**2)))
gain_coefficient = alpha_op * neighbourhood_func
learning_rate_multiplier = gain_coefficient.repeat(3, 1).T # copy to every dimension
learning_rate_multiplier

In [ ]:
colors[4] - som.weights

In [ ]:
torch.mul(learning_rate_multiplier, colors[4] - som.weights)

In [ ]:
x = colors[4]
#torch.linalg.vector_norm(x - som.weights, ord=2, dim=0)
[np.linalg.norm(x-som.weights[i]) for i in range(som.n_neurons)]

In [ ]:
np.linalg.norm(x-som.weights[0])

In [ ]:
torch.linalg.vector_norm(x - som.weights[0], ord=2)

In [ ]:
torch.linalg.vector_norm(x - som.weights, ord=2, dim=1)

In [ ]:
torch.linalg.vector_norm(x - som.weights, ord=2, dim=1).min(0)

## Triangle experiment

Let's reproduce an experiment from [Kohonen's paper](https://sci2s.ugr.es/keel/pdf/algorithm/articulo/1990-Kohonen-PIEEE.pdf). It's about random dots that become organized as a curve that evenly takes the whole space.

In [ ]:
# v_x, v_y, v_z \in [0,1]
def vector_to_barycentric_coord(v):
    v /= v.sum(dim=1, keepdim=True)
    return v

In [ ]:
# Generate vertices of an equilateral triangle
triangle_side = 1.0
half_height = np.sqrt(3) / 2 * triangle_side
triangle_points = np.array([[0, 0], 
                            [triangle_side, 0], 
                            [triangle_side / 2, half_height]])

def plot_triangle_and_data(data):
    # Plotting the equilateral triangle
    plt.figure(figsize=(6, 6))
    plt.fill(triangle_points[:, 0], triangle_points[:, 1], 'b', alpha=0.3, label="Equilateral Triangle")

    # Plotting the random vectors inside the triangle
    origin = np.array([0, 0])  # origin point for vectors
    line_segments_x = []
    line_segments_y = []
    for i in range(data.shape[0]):
        vec = data[i].numpy()
        if i < data.shape[0] - 1:
            next_vec = data[i+1].numpy()
            line_segments_x.append(vec[0])
            line_segments_x.append(next_vec[0])
            line_segments_y.append(vec[1])
            line_segments_y.append(next_vec[1])
        # plt.plot(vec, next_vec, color='r')
        # bold points
        #plt.plot(vec[0], vec[1], 'bo') 
        # vector line
        #plt.quiver(origin[0], origin[1], vec[0], vec[1], angles='xy', scale_units='xy', scale=1, color='g', alpha=0.6)

    # connection between one point and the next
    plt.plot(line_segments_x, line_segments_y, color='r')

    # Set plot limits and labels
    plt.xlim(-0.25, 1.25)
    plt.ylim(-0.25, 1.25)
    #plt.gca().set_aspect('equal', adjustable='box')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    #plt.legend()
    plt.title('Equilateral Triangle and Random Vectors Inside')
    plt.grid(True)
    plt.show()

The function `random_barycentric_coordinates` generates 3 random weights for each point and normalizes them so that they sum to 1.

In [ ]:
def random_barycentric_coordinates(n_points):
    barycentric_coords = torch.rand(n_points, 3)
    barycentric_coords /= barycentric_coords.sum(dim=1, keepdim=True)
    return barycentric_coords

Just 10 random points in a equilateral triangle. First, a 3D vector created from three uniformly distributed values in a single unit cube. Then they are converted into 2D points (blue balls) (or we can think about them as 2D vectors as depicted by a green arrows but that's all fiction hehe). This conversion uses barycentric coordinate system and thus all 3D values from [0,1] interval lay inside the triangle.

In [ ]:

# Generate 10 random points inside the triangle using barycentric coordinates
n_points = 100
bary_coords = random_barycentric_coordinates(n_points)

# bary_coords = torch.tensor([
#     [0.5,0.5,0],
#     [0.33,0.33,0.33],
#     [0,0,1],
# ], dtype=torch.float32)

# Convert barycentric coordinates to 2D points inside the triangle
vectors_in_triangle = torch.matmul(bary_coords, torch.tensor(triangle_points).float())



In [ ]:
vectors_in_triangle.shape
plot_triangle_and_data(vectors_in_triangle)

In [ ]:
bary_coords

In [ ]:

n_points = 100

m = 100
n = 1
n_iter = 5000
som = SOM(m, n, 3)


for iter_no in range(n_iter):
    bary_coords = random_barycentric_coordinates(n_points)
    data = bary_coords
    #Train with each vector one by one
    for i in range(len(data)):
        learning_rate_op = 1.0 - iter_no/(1.0*n_iter)
        som(data[i], learning_rate_op)


In [ ]:
w = som.get_weights()
points_in_triangle = torch.matmul(w, torch.tensor(triangle_points).float())

In [ ]:
plot_triangle_and_data(points_in_triangle)

In [ ]:

#Store a centroid grid for easy retrieval later on
weights = som.get_weights()
locations = som.get_locations()

#Get output grid
image_grid = centroid_grid

#Map colours to their closest neurons
mapped = som.map_vects(torch.Tensor(colors))

#Plot
plt.imshow(image_grid)
plt.title('Color SOM')
plt.show()